## BaarleNet [Part 3 - Simplify the Geometry]


In [1]:
import numpy as np
import torch.nn as nn
import torch
import matplotlib.pyplot as plt
import torch.optim as optim
import cv2
from matplotlib.gridspec import GridSpec
from tqdm import tqdm
import os

class BaarleNet(nn.Module):
    def __init__(self, hidden_layers=[64]):
        super(BaarleNet, self).__init__()
        layers = [nn.Linear(2, hidden_layers[0]), nn.ReLU()]
        for i in range(len(hidden_layers)-1):
            layers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            layers.append(nn.ReLU())
        layers.append(nn.Linear(hidden_layers[-1], 2))
        self.layers=layers
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

model=BaarleNet([2])

In [2]:
model.layers[0].weight

Parameter containing:
tensor([[-0.1127,  0.5813],
        [ 0.3683,  0.6754]], requires_grad=True)

In [3]:
model.layers[0].bias

Parameter containing:
tensor([0.6403, 0.5566], requires_grad=True)

In [4]:
model.layers[2].weight

Parameter containing:
tensor([[-0.0716,  0.3243],
        [ 0.5679,  0.5450]], requires_grad=True)

In [5]:
model.layers[2].bias

Parameter containing:
tensor([0.5462, 0.3574], requires_grad=True)

In [6]:
# 2-Hidden Layer Neural Network Visualization
# Shows how ReLU layers stack to create complex partitions

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.patches as patches
from mpl_toolkits.axes_grid1 import make_axes_locatable

class TwoLayerNeuralNetworkVisualizer:
    def __init__(self):
        self.output = widgets.Output()
        self.setup_widgets()
        
    def setup_widgets(self):
        style = {'description_width': '140px'}
        layout = widgets.Layout(width='300px')
        
        # Layer 1 (Input -> Hidden1)
        self.l1_w11_slider = widgets.FloatSlider(
            value=1.0, min=-3.0, max=3.0, step=0.1,
            description='L1: w₁₁ (N1):', style=style, layout=layout
        )
        self.l1_w12_slider = widgets.FloatSlider(
            value=1.0, min=-3.0, max=3.0, step=0.1,
            description='L1: w₁₂ (N1):', style=style, layout=layout
        )
        self.l1_b1_slider = widgets.FloatSlider(
            value=0.0, min=-3.0, max=3.0, step=0.1,
            description='L1: b₁ (N1):', style=style, layout=layout
        )
        
        self.l1_w21_slider = widgets.FloatSlider(
            value=-1.0, min=-3.0, max=3.0, step=0.1,
            description='L1: w₂₁ (N2):', style=style, layout=layout
        )
        self.l1_w22_slider = widgets.FloatSlider(
            value=1.0, min=-3.0, max=3.0, step=0.1,
            description='L1: w₂₂ (N2):', style=style, layout=layout
        )
        self.l1_b2_slider = widgets.FloatSlider(
            value=0.0, min=-3.0, max=3.0, step=0.1,
            description='L1: b₂ (N2):', style=style, layout=layout
        )
        
        # Layer 2 (Hidden1 -> Hidden2)
        self.l2_w11_slider = widgets.FloatSlider(
            value=1.0, min=-3.0, max=3.0, step=0.1,
            description='L2: w₁₁ (N1):', style=style, layout=layout
        )
        self.l2_w12_slider = widgets.FloatSlider(
            value=0.5, min=-3.0, max=3.0, step=0.1,
            description='L2: w₁₂ (N1):', style=style, layout=layout
        )
        self.l2_b1_slider = widgets.FloatSlider(
            value=0.0, min=-3.0, max=3.0, step=0.1,
            description='L2: b₁ (N1):', style=style, layout=layout
        )
        
        self.l2_w21_slider = widgets.FloatSlider(
            value=0.5, min=-3.0, max=3.0, step=0.1,
            description='L2: w₂₁ (N2):', style=style, layout=layout
        )
        self.l2_w22_slider = widgets.FloatSlider(
            value=1.0, min=-3.0, max=3.0, step=0.1,
            description='L2: w₂₂ (N2):', style=style, layout=layout
        )
        self.l2_b2_slider = widgets.FloatSlider(
            value=-0.5, min=-3.0, max=3.0, step=0.1,
            description='L2: b₂ (N2):', style=style, layout=layout
        )
        
        # Visualization options
        self.viz_mode = widgets.RadioButtons(
            options=['Layer 1 Only', 'Layer 2 Only', 'Both Layers', 'Activations'],
            value='Both Layers',
            description='Show:',
            style={'description_width': 'initial'}
        )
        
        self.show_boundaries = widgets.Checkbox(
            value=True,
            description='Show decision boundaries',
            style={'description_width': 'initial'}
        )
        
        self.show_equations = widgets.Checkbox(
            value=True,
            description='Show region equations',
            style={'description_width': 'initial'}
        )
        
        # Control buttons
        self.reset_button = widgets.Button(
            description='Reset to Defaults',
            button_style='info',
            layout=widgets.Layout(width='200px')
        )
        
        self.auto_update = widgets.Checkbox(
            value=True,
            description='Auto-update',
            style={'description_width': 'initial'}
        )
        
        self.update_button = widgets.Button(
            description='Update Plot',
            button_style='success',
            layout=widgets.Layout(width='120px')
        )
        
        # Group controls
        self.layer1_box = widgets.VBox([
            widgets.HTML('<h3 style="color: #e74c3c; margin: 5px 0;">🔴 Layer 1: Input → Hidden1</h3>'),
            widgets.HTML('<div style="font-family: monospace; background: #fdf2f2; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 10px;">h1₁ = ReLU(w₁₁×x₁ + w₁₂×x₂ + b₁)<br>h1₂ = ReLU(w₂₁×x₁ + w₂₂×x₂ + b₂)</div>'),
            self.l1_w11_slider,
            self.l1_w12_slider,
            self.l1_b1_slider,
            widgets.HTML('<hr style="margin: 8px 0;">'),
            self.l1_w21_slider,
            self.l1_w22_slider,
            self.l1_b2_slider
        ])
        
        self.layer2_box = widgets.VBox([
            widgets.HTML('<h3 style="color: #3498db; margin: 5px 0;">🔵 Layer 2: Hidden1 → Hidden2</h3>'),
            widgets.HTML('<div style="font-family: monospace; background: #f0f7ff; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 10px;">h2₁ = ReLU(w₁₁×h1₁ + w₁₂×h1₂ + b₁)<br>h2₂ = ReLU(w₂₁×h1₁ + w₂₂×h1₂ + b₂)</div>'),
            self.l2_w11_slider,
            self.l2_w12_slider,
            self.l2_b1_slider,
            widgets.HTML('<hr style="margin: 8px 0;">'),
            self.l2_w21_slider,
            self.l2_w22_slider,
            self.l2_b2_slider
        ])
        
        self.options_box = widgets.VBox([
            widgets.HTML('<h3 style="color: #9b59b6; margin: 5px 0;">⚙️ Visualization Options</h3>'),
            self.viz_mode,
            widgets.HTML('<br>'),
            self.show_boundaries,
            self.show_equations,
            widgets.HTML('<br>'),
            self.auto_update,
            self.update_button,
            self.reset_button
        ])
        
        # Bind events
        all_sliders = [
            self.l1_w11_slider, self.l1_w12_slider, self.l1_b1_slider,
            self.l1_w21_slider, self.l1_w22_slider, self.l1_b2_slider,
            self.l2_w11_slider, self.l2_w12_slider, self.l2_b1_slider,
            self.l2_w21_slider, self.l2_w22_slider, self.l2_b2_slider
        ]
        
        for slider in all_sliders:
            slider.observe(self.on_value_change, names='value')
        
        for widget in [self.viz_mode, self.show_boundaries, self.show_equations]:
            widget.observe(self.on_value_change, names='value')
        
        self.reset_button.on_click(self.reset_values)
        self.update_button.on_click(self.manual_update)
        
    def relu(self, x):
        return np.maximum(0, x)
    
    def on_value_change(self, change):
        if self.auto_update.value:
            self.update_plot()
    
    def manual_update(self, button=None):
        self.update_plot()
    
    def compute_network(self, X1, X2):
        """Compute forward pass through the 2-layer network"""
        # Layer 1: Input -> Hidden1
        z1_1 = self.l1_w11_slider.value * X1 + self.l1_w12_slider.value * X2 + self.l1_b1_slider.value
        z1_2 = self.l1_w21_slider.value * X1 + self.l1_w22_slider.value * X2 + self.l1_b2_slider.value
        
        h1_1 = self.relu(z1_1)  # First neuron of layer 1
        h1_2 = self.relu(z1_2)  # Second neuron of layer 1
        
        # Layer 2: Hidden1 -> Hidden2
        z2_1 = self.l2_w11_slider.value * h1_1 + self.l2_w12_slider.value * h1_2 + self.l2_b1_slider.value
        z2_2 = self.l2_w21_slider.value * h1_1 + self.l2_w22_slider.value * h1_2 + self.l2_b2_slider.value
        
        h2_1 = self.relu(z2_1)  # First neuron of layer 2
        h2_2 = self.relu(z2_2)  # Second neuron of layer 2
        
        return {
            'z1': (z1_1, z1_2), 'h1': (h1_1, h1_2),
            'z2': (z2_1, z2_2), 'h2': (h2_1, h2_2)
        }
    
    def update_plot(self):
        with self.output:
            clear_output(wait=True)
            
            # Create grid
            x1 = np.linspace(-1, 1, 200)
            x2 = np.linspace(-1, 1, 200)
            X1, X2 = np.meshgrid(x1, x2)
            
            # Compute network outputs
            outputs = self.compute_network(X1, X2)
            
            # Create plots based on visualization mode
            if self.viz_mode.value == 'Both Layers':
                fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 9))
                self.plot_layer1_regions(ax1, X1, X2, outputs)
                self.plot_layer2_regions(ax2, X1, X2, outputs)
            elif self.viz_mode.value == 'Layer 1 Only':
                fig, ax = plt.subplots(1, 1, figsize=(12, 10))
                self.plot_layer1_regions(ax, X1, X2, outputs)
            elif self.viz_mode.value == 'Layer 2 Only':
                fig, ax = plt.subplots(1, 1, figsize=(12, 10))
                self.plot_layer2_regions(ax, X1, X2, outputs)
            elif self.viz_mode.value == 'Activations':
                fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(18, 16))
                self.plot_activation_heatmaps(ax1, ax2, ax3, ax4, X1, X2, outputs)
            
            plt.tight_layout()
            plt.show()
    
    def plot_layer1_regions(self, ax, X1, X2, outputs):
        """Plot Layer 1 activation regions"""
        h1_1, h1_2 = outputs['h1']
        
        # Create region map for Layer 1
        region_map = np.zeros_like(X1)
        mask1 = (h1_1 > 0) & (h1_2 == 0)  # Only neuron 1 active
        mask2 = (h1_1 == 0) & (h1_2 > 0)  # Only neuron 2 active  
        mask3 = (h1_1 > 0) & (h1_2 > 0)   # Both neurons active
        
        region_map[mask1] = 1
        region_map[mask2] = 2
        region_map[mask3] = 3
        
        # Plot regions
        colors = ['#f8f9fa', '#ffcccc', '#ccffff', '#ffccff']
        cmap = LinearSegmentedColormap.from_list('layer1', colors, N=4)
        im = ax.imshow(region_map, extent=[-1, 1, -1, 1], cmap=cmap, alpha=0.8, origin='lower')
        
        # Decision boundaries
        if self.show_boundaries.value:
            self.plot_decision_boundary(ax, 
                self.l1_w11_slider.value, self.l1_w12_slider.value, self.l1_b1_slider.value, 
                'red', 'L1 N1 boundary', linewidth=3)
            self.plot_decision_boundary(ax, 
                self.l1_w21_slider.value, self.l1_w22_slider.value, self.l1_b2_slider.value, 
                'cyan', 'L1 N2 boundary', linewidth=3)
        
        # Region labels
        if self.show_equations.value:
            ax.text(-0.7, -0.7, 'h1₁=0\nh1₂=0', bbox=dict(boxstyle="round", facecolor="white", alpha=0.9), fontsize=9, ha='center')
            ax.text(0.6, -0.6, 'h1₁>0\nh1₂=0', bbox=dict(boxstyle="round", facecolor="#ffcccc", alpha=0.9), fontsize=9, ha='center')
            ax.text(-0.6, 0.6, 'h1₁=0\nh1₂>0', bbox=dict(boxstyle="round", facecolor="#ccffff", alpha=0.9), fontsize=9, ha='center')
            ax.text(0.5, 0.5, 'h1₁>0\nh1₂>0', bbox=dict(boxstyle="round", facecolor="#ffccff", alpha=0.9), fontsize=9, ha='center')
        
        self.format_axis(ax, 'Layer 1: Input Space Partitioning\n(4 linear regions created by 2 ReLU neurons)')
        
        # Legend
        legend_elements = [
            patches.Patch(color='#ffcccc', label='h1₁>0, h1₂=0'),
            patches.Patch(color='#ccffff', label='h1₁=0, h1₂>0'),
            patches.Patch(color='#ffccff', label='h1₁>0, h1₂>0'),
            patches.Patch(color='#f8f9fa', label='h1₁=0, h1₂=0')
        ]
        ax.legend(handles=legend_elements, loc='upper left', framealpha=0.95)
    
    def plot_layer2_regions(self, ax, X1, X2, outputs):
        """Plot Layer 2 activation regions (operating on Layer 1 outputs)"""
        h2_1, h2_2 = outputs['h2']
        
        # Create region map for Layer 2
        region_map = np.zeros_like(X1)
        mask1 = (h2_1 > 0) & (h2_2 == 0)  # Only neuron 1 active
        mask2 = (h2_1 == 0) & (h2_2 > 0)  # Only neuron 2 active  
        mask3 = (h2_1 > 0) & (h2_2 > 0)   # Both neurons active
        
        region_map[mask1] = 1
        region_map[mask2] = 2
        region_map[mask3] = 3
        
        # Plot regions
        colors = ['#f0f0f0', '#ffe6e6', '#e6f3ff', '#f0e6ff']
        cmap = LinearSegmentedColormap.from_list('layer2', colors, N=4)
        im = ax.imshow(region_map, extent=[-1, 1, -1, 1], cmap=cmap, alpha=0.9, origin='lower')
        
        # Show Layer 1 boundaries (faded)
        if self.show_boundaries.value:
            self.plot_decision_boundary(ax, 
                self.l1_w11_slider.value, self.l1_w12_slider.value, self.l1_b1_slider.value, 
                'red', 'L1 boundaries', linewidth=2, alpha=0.4, linestyle='--')
            self.plot_decision_boundary(ax, 
                self.l1_w21_slider.value, self.l1_w22_slider.value, self.l1_b2_slider.value, 
                'cyan', '', linewidth=2, alpha=0.4, linestyle='--')
            
            # Layer 2 decision boundaries are more complex - they're curves!
            self.plot_layer2_boundaries(ax, X1, X2, outputs)
        
        self.format_axis(ax, 'Layer 2: Complex Partitioning\n(Non-linear boundaries created by ReLU on ReLU)')
        
        # Legend
        legend_elements = [
            patches.Patch(color='#ffe6e6', label='h2₁>0, h2₂=0'),
            patches.Patch(color='#e6f3ff', label='h2₁=0, h2₂>0'),
            patches.Patch(color='#f0e6ff', label='h2₁>0, h2₂>0'),
            patches.Patch(color='#f0f0f0', label='h2₁=0, h2₂=0')
        ]
        ax.legend(handles=legend_elements, loc='upper left', framealpha=0.95)
    
    def plot_layer2_boundaries(self, ax, X1, X2, outputs):
        """Plot Layer 2 decision boundaries (which can be curved!)"""
        z2_1, z2_2 = outputs['z2']
        
        # Layer 2 boundaries are where z2_i = 0, but z2_i is a function of h1, which is piecewise linear
        # This creates piecewise linear boundaries in the h1 space, but potentially curved in input space
        
        # Plot contours where z2 = 0
        try:
            cs1 = ax.contour(X1, X2, z2_1, levels=[0], colors=['darkred'], linewidths=3, alpha=0.8)
            cs2 = ax.contour(X1, X2, z2_2, levels=[0], colors=['darkblue'], linewidths=3, alpha=0.8)
            
            # Add labels
            if len(cs1.collections) > 0:
                ax.clabel(cs1, inline=True, fontsize=8, fmt='L2 N1=0')
            if len(cs2.collections) > 0:
                ax.clabel(cs2, inline=True, fontsize=8, fmt='L2 N2=0')
        except:
            pass  # Skip if contour fails
    
    def plot_activation_heatmaps(self, ax1, ax2, ax3, ax4, X1, X2, outputs):
        """Plot heatmaps of individual neuron activations"""
        h1_1, h1_2 = outputs['h1']
        h2_1, h2_2 = outputs['h2']
        
        # Layer 1 activations
        im1 = ax1.imshow(h1_1, extent=[-1, 1, -1, 1], cmap='Reds', origin='lower')
        ax1.set_title('Layer 1, Neuron 1 Activation', fontsize=12)
        plt.colorbar(im1, ax=ax1, fraction=0.046, pad=0.04)
        
        im2 = ax2.imshow(h1_2, extent=[-1, 1, -1, 1], cmap='Blues', origin='lower')
        ax2.set_title('Layer 1, Neuron 2 Activation', fontsize=12)
        plt.colorbar(im2, ax=ax2, fraction=0.046, pad=0.04)
        
        # Layer 2 activations
        im3 = ax3.imshow(h2_1, extent=[-1, 1, -1, 1], cmap='Greens', origin='lower')
        ax3.set_title('Layer 2, Neuron 1 Activation', fontsize=12)
        plt.colorbar(im3, ax=ax3, fraction=0.046, pad=0.04)
        
        im4 = ax4.imshow(h2_2, extent=[-1, 1, -1, 1], cmap='Purples', origin='lower')
        ax4.set_title('Layer 2, Neuron 2 Activation', fontsize=12)
        plt.colorbar(im4, ax=ax4, fraction=0.046, pad=0.04)
        
        for ax in [ax1, ax2, ax3, ax4]:
            ax.set_xlabel('x₁')
            ax.set_ylabel('x₂')
            ax.grid(True, alpha=0.3)
            ax.set_aspect('equal')
    
    def plot_decision_boundary(self, ax, w1, w2, b, color, label, linewidth=3, alpha=0.9, linestyle='-'):
        """Plot decision boundary where w1*x1 + w2*x2 + b = 0"""
        if abs(w2) > 1e-6:
            x1_vals = np.linspace(-1, 1, 100)
            x2_vals = -(w1 * x1_vals + b) / w2
            mask = (x2_vals >= -1) & (x2_vals <= 1)
            if np.any(mask):
                ax.plot(x1_vals[mask], x2_vals[mask], 
                       color=color, linewidth=linewidth, label=label, alpha=alpha, linestyle=linestyle)
        elif abs(w1) > 1e-6:
            x1_boundary = -b / w1
            if -1 <= x1_boundary <= 1:
                ax.axvline(x1_boundary, color=color, linewidth=linewidth, 
                          label=label, alpha=alpha, linestyle=linestyle)
    
    def format_axis(self, ax, title):
        ax.set_xlim(-1, 1)
        ax.set_ylim(-1, 1)
        ax.set_xlabel('x₁ (Input 1)', fontsize=12)
        ax.set_ylabel('x₂ (Input 2)', fontsize=12)
        ax.set_title(title, fontsize=14, pad=15)
        ax.grid(True, alpha=0.3)
        ax.set_aspect('equal')
    
    def reset_values(self, button):
        # Layer 1 defaults
        self.l1_w11_slider.value = 1.0
        self.l1_w12_slider.value = 1.0
        self.l1_b1_slider.value = 0.0
        self.l1_w21_slider.value = -1.0
        self.l1_w22_slider.value = 1.0
        self.l1_b2_slider.value = 0.0
        
        # Layer 2 defaults
        self.l2_w11_slider.value = 1.0
        self.l2_w12_slider.value = 0.5
        self.l2_b1_slider.value = 0.0
        self.l2_w21_slider.value = 0.5
        self.l2_w22_slider.value = 1.0
        self.l2_b2_slider.value = -0.5
        
    def display(self):
        controls = widgets.HBox([
            self.layer1_box,
            self.layer2_box,
            self.options_box
        ], layout=widgets.Layout(justify_content='space-around'))
        
        display(widgets.VBox([
            widgets.HTML('<h2 style="text-align: center; color: #2c3e50; margin-bottom: 15px;">🧠 2-Layer Neural Network: How ReLUs Stack</h2>'),
            controls,
            widgets.HTML('<hr style="margin: 15px 0;">'),
            self.output
        ]))
        
        self.update_plot()

# Create and display the visualizer
print("🚀 Creating 2-Layer Neural Network Visualizer...")
visualizer = TwoLayerNeuralNetworkVisualizer()
visualizer.display()

print("""
🧠 HOW MULTIPLE ReLU LAYERS WORK TOGETHER:

🔴 LAYER 1: Creates 4 LINEAR regions in input space
• Each ReLU neuron creates a half-plane
• 2 neurons → up to 4 regions possible

🔵 LAYER 2: Operates on Layer 1's piecewise-linear output
• Takes (h1₁, h1₂) as input (which varies across regions)
• Each neuron creates boundaries in the (h1₁, h1₂) space
• These become CURVED boundaries in original (x₁, x₂) space!

🎯 KEY INSIGHTS:
1. Layer 1: Linear boundaries in input space
2. Layer 2: Can create curved boundaries in input space!
3. Each layer increases representation complexity
4. More layers → more complex decision boundaries

🎛️ EXPERIMENTS TO TRY:
1. Start with 'Layer 1 Only' → see simple linear regions
2. Switch to 'Layer 2 Only' → see complex curved boundaries  
3. Use 'Activations' mode → see how each neuron responds
4. Try different Layer 2 weights → see boundary curvature change

💡 This is how deep networks build complex functions: each ReLU layer can create more intricate partitions than the previous layer!
""")

🚀 Creating 2-Layer Neural Network Visualizer...



🧠 HOW MULTIPLE ReLU LAYERS WORK TOGETHER:

🔴 LAYER 1: Creates 4 LINEAR regions in input space
• Each ReLU neuron creates a half-plane
• 2 neurons → up to 4 regions possible

🔵 LAYER 2: Operates on Layer 1's piecewise-linear output
• Takes (h1₁, h1₂) as input (which varies across regions)
• Each neuron creates boundaries in the (h1₁, h1₂) space
• These become CURVED boundaries in original (x₁, x₂) space!

🎯 KEY INSIGHTS:
1. Layer 1: Linear boundaries in input space
2. Layer 2: Can create curved boundaries in input space!
3. Each layer increases representation complexity
4. More layers → more complex decision boundaries

🎛️ EXPERIMENTS TO TRY:
1. Start with 'Layer 1 Only' → see simple linear regions
2. Switch to 'Layer 2 Only' → see complex curved boundaries  
3. Use 'Activations' mode → see how each neuron responds
4. Try different Layer 2 weights → see boundary curvature change

💡 This is how deep networks build complex functions: each ReLU layer can create more intricate partitions t

In [10]:
# # Neural Network Input Space Partitioning + Output Layer Visualization
# # Shows how the output layer operates on hidden layer activations

# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import LinearSegmentedColormap
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import matplotlib.patches as patches
# from mpl_toolkits.axes_grid1 import make_axes_locatable

# class NeuralNetworkVisualizer:
#     def __init__(self):
#         self.output = widgets.Output()
#         self.setup_widgets()
        
#     def setup_widgets(self):
#         # Create sliders for weights and biases
#         style = {'description_width': '130px'}
#         layout = widgets.Layout(width='300px')
        
#         # Hidden layer (Neuron 1 & 2) controls
#         self.w11_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₁₁ (Hidden):', style=style, layout=layout
#         )
#         self.w12_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₁₂ (Hidden):', style=style, layout=layout
#         )
#         self.b1_slider = widgets.FloatSlider(
#             value=0.0, min=-3.0, max=3.0, step=0.1,
#             description='b₁ (Hidden):', style=style, layout=layout
#         )
        
#         self.w21_slider = widgets.FloatSlider(
#             value=-1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₂₁ (Hidden):', style=style, layout=layout
#         )
#         self.w22_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₂₂ (Hidden):', style=style, layout=layout
#         )
#         self.b2_slider = widgets.FloatSlider(
#             value=0.0, min=-3.0, max=3.0, step=0.1,
#             description='b₂ (Hidden):', style=style, layout=layout
#         )
        
#         # Output layer controls
#         self.w_out1_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w_out1 (Output):', style=style, layout=layout
#         )
#         self.w_out2_slider = widgets.FloatSlider(
#             value=-1.0, min=-3.0, max=3.0, step=0.1,
#             description='w_out2 (Output):', style=style, layout=layout
#         )
#         self.b_out_slider = widgets.FloatSlider(
#             value=0.0, min=-3.0, max=3.0, step=0.1,
#             description='b_out (Output):', style=style, layout=layout
#         )
        
#         # Visualization mode
#         self.viz_mode = widgets.RadioButtons(
#             options=['Hidden Layer Regions', 'Output Values', 'Both'],
#             value='Both',
#             description='Show:',
#             style={'description_width': 'initial'}
#         )
        
#         # Reset button
#         self.reset_button = widgets.Button(
#             description='Reset to Defaults',
#             button_style='info',
#             layout=widgets.Layout(width='200px')
#         )
        
#         # Auto-update checkbox
#         self.auto_update = widgets.Checkbox(
#             value=True,
#             description='Auto-update plot',
#             style={'description_width': 'initial'}
#         )
        
#         # Manual update button
#         self.update_button = widgets.Button(
#             description='Update Plot',
#             button_style='success',
#             layout=widgets.Layout(width='150px')
#         )
        
#         # Group controls
#         self.hidden_layer_box = widgets.VBox([
#             widgets.HTML('<h3 style="color: #2c3e50; margin: 5px 0;">🔄 Hidden Layer (2 neurons)</h3>'),
#             widgets.HTML('<div style="font-family: monospace; background: #e8f4f8; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 11px;">z₁ = w₁₁×x₁ + w₁₂×x₂ + b₁ → a₁ = ReLU(z₁)<br>z₂ = w₂₁×x₁ + w₂₂×x₂ + b₂ → a₂ = ReLU(z₂)</div>'),
#             self.w11_slider,
#             self.w12_slider,
#             self.b1_slider,
#             widgets.HTML('<hr style="margin: 10px 0;">'),
#             self.w21_slider,
#             self.w22_slider,
#             self.b2_slider
#         ])
        
#         self.output_layer_box = widgets.VBox([
#             widgets.HTML('<h3 style="color: #e74c3c; margin: 5px 0;">🎯 Output Layer (1 neuron)</h3>'),
#             widgets.HTML('<div style="font-family: monospace; background: #fdf2f2; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 11px;">y = w_out1×a₁ + w_out2×a₂ + b_out<br><br>⚠️ SAME operation applied everywhere!<br>Different results due to different a₁, a₂</div>'),
#             self.w_out1_slider,
#             self.w_out2_slider,
#             self.b_out_slider,
#             widgets.HTML('<hr style="margin: 10px 0;">'),
#             self.viz_mode
#         ])
        
#         # Bind events
#         all_sliders = [self.w11_slider, self.w12_slider, self.b1_slider,
#                       self.w21_slider, self.w22_slider, self.b2_slider,
#                       self.w_out1_slider, self.w_out2_slider, self.b_out_slider]
        
#         for slider in all_sliders:
#             slider.observe(self.on_value_change, names='value')
        
#         self.viz_mode.observe(self.on_value_change, names='value')
#         self.reset_button.on_click(self.reset_values)
#         self.update_button.on_click(self.manual_update)
        
#     def relu(self, x):
#         return np.maximum(0, x)
    
#     def on_value_change(self, change):
#         if self.auto_update.value:
#             self.update_plot()
    
#     def manual_update(self, button=None):
#         self.update_plot()
    
#     def update_plot(self):
#         with self.output:
#             clear_output(wait=True)
            
#             # Get current values
#             w11, w12, b1 = self.w11_slider.value, self.w12_slider.value, self.b1_slider.value
#             w21, w22, b2 = self.w21_slider.value, self.w22_slider.value, self.b2_slider.value
#             w_out1, w_out2, b_out = self.w_out1_slider.value, self.w_out2_slider.value, self.b_out_slider.value
            
#             # Create figure with subplots
#             if self.viz_mode.value == 'Both':
#                 fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(18, 8))
#             else:
#                 fig, ax1 = plt.subplots(1, 1, figsize=(12, 10))
#                 ax2 = None
            
#             # Create grid
#             x1 = np.linspace(-1, 1, 300)
#             x2 = np.linspace(-1, 1, 300)
#             X1, X2 = np.meshgrid(x1, x2)
            
#             # Calculate hidden layer outputs
#             z1 = w11 * X1 + w12 * X2 + b1
#             z2 = w21 * X1 + w22 * X2 + b2
#             a1 = self.relu(z1)
#             a2 = self.relu(z2)
            
#             # Calculate output layer
#             y_output = w_out1 * a1 + w_out2 * a2 + b_out
            
#             # Plot based on mode
#             if self.viz_mode.value in ['Hidden Layer Regions', 'Both']:
#                 self.plot_hidden_regions(ax1, X1, X2, a1, a2, w11, w12, b1, w21, w22, b2, w_out1, w_out2, b_out)
            
#             if self.viz_mode.value in ['Output Values', 'Both']:
#                 target_ax = ax2 if ax2 is not None else ax1
#                 self.plot_output_values(target_ax, X1, X2, y_output, a1, a2, w11, w12, b1, w21, w22, b2, w_out1, w_out2, b_out)
            
#             plt.tight_layout()
#             plt.show()
    
#     def plot_hidden_regions(self, ax, X1, X2, a1, a2, w11, w12, b1, w21, w22, b2, w_out1, w_out2, b_out):
#         """Plot the hidden layer activation regions"""
#         # Create region map
#         region_map = np.zeros_like(X1)
        
#         # Define regions based on which neurons are active
#         mask1 = (a1 > 0) & (a2 == 0)  # Only neuron 1 active
#         mask2 = (a1 == 0) & (a2 > 0)  # Only neuron 2 active  
#         mask3 = (a1 > 0) & (a2 > 0)   # Both neurons active
        
#         region_map[mask1] = 1
#         region_map[mask2] = 2
#         region_map[mask3] = 3
        
#         # Custom colormap
#         colors = ['#f8f9fa', '#ffb3b3', '#b3d9d9', '#d9b3ff']
#         cmap = LinearSegmentedColormap.from_list('custom', colors, N=4)
        
#         # Plot regions
#         im = ax.imshow(region_map, extent=[-1, 1, -1, 1], 
#                       cmap=cmap, alpha=0.8, origin='lower')
        
#         # Plot decision boundaries
#         self.plot_decision_boundary(ax, w11, w12, b1, 'red', 'Neuron 1 boundary', linewidth=3)
#         self.plot_decision_boundary(ax, w21, w22, b2, 'cyan', 'Neuron 2 boundary', linewidth=3)
        
#         # Add region labels with output formulas
#         self.add_region_labels(ax, w_out1, w_out2, b_out)
        
#         # Formatting
#         ax.set_xlim(-1, 1)
#         ax.set_ylim(-1, 1)
#         ax.set_xlabel('x₁ (Input 1)', fontsize=12)
#         ax.set_ylabel('x₂ (Input 2)', fontsize=12)
#         ax.set_title('Hidden Layer Activation Regions\n(Same output formula applied to each region)', fontsize=14, pad=20)
#         ax.grid(True, alpha=0.3, linewidth=0.5)
#         ax.set_aspect('equal')
        
#         # Legend
#         legend_elements = [
#             patches.Patch(color='#ffb3b3', label='Region 1: a₁>0, a₂=0'),
#             patches.Patch(color='#b3d9d9', label='Region 2: a₁=0, a₂>0'),
#             patches.Patch(color='#d9b3ff', label='Region 3: a₁>0, a₂>0'),
#             patches.Patch(color='#f8f9fa', label='Region 0: a₁=0, a₂=0')
#         ]
#         ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(0.0, 1.0), framealpha=0.95)
    
#     def plot_output_values(self, ax, X1, X2, y_output, a1, a2, w11, w12, b1, w21, w22, b2, w_out1, w_out2, b_out):
#         """Plot the final output values"""
#         # Create a colormap for output values
#         vmin, vmax = np.min(y_output), np.max(y_output)
        
#         # Plot output as heatmap
#         im = ax.contourf(X1, X2, y_output, levels=50, cmap='RdYlBu_r', alpha=0.9)
        
#         # Add colorbar
#         divider = make_axes_locatable(ax)
#         cax = divider.append_axes("right", size="5%", pad=0.1)
#         cbar = plt.colorbar(im, cax=cax)
#         cbar.set_label('Output Value', fontsize=12)
        
#         # Overlay contour lines
#         contour = ax.contour(X1, X2, y_output, levels=10, colors='black', alpha=0.6, linewidths=1)
#         ax.clabel(contour, inline=True, fontsize=8, fmt='%.1f')
        
#         # Plot decision boundaries from hidden layer
#         self.plot_decision_boundary(ax, w11, w12, b1, 'red', 'Hidden boundary 1', linewidth=2, alpha=0.8)
#         self.plot_decision_boundary(ax, w21, w22, b2, 'cyan', 'Hidden boundary 2', linewidth=2, alpha=0.8)
        
#         # Formatting
#         ax.set_xlim(-1, 1)
#         ax.set_ylim(-1, 1)
#         ax.set_xlabel('x₁ (Input 1)', fontsize=12)
#         ax.set_ylabel('x₂ (Input 2)', fontsize=12)
#         ax.set_title(f'Final Output: y = {w_out1:.1f}×a₁ + {w_out2:.1f}×a₂ + {b_out:.1f}\n(Piecewise linear function)', fontsize=14, pad=20)
#         ax.grid(True, alpha=0.3, linewidth=0.5)
#         ax.set_aspect('equal')
    
#     def add_region_labels(self, ax, w_out1, w_out2, b_out):
#         """Add labels showing the output formula in each region"""
#         # Region 0: a1=0, a2=0
#         y0 = b_out
#         ax.text(-0.8, -0.8, f'Region 0:\ny = {y0:.1f}', 
#                bbox=dict(boxstyle="round,pad=0.3", facecolor="white", alpha=0.9),
#                fontsize=10, ha='center')
        
#         # Region 1: a1>0, a2=0  
#         ax.text(0.7, -0.7, f'Region 1:\ny = {w_out1:.1f}×a₁ + {b_out:.1f}', 
#                bbox=dict(boxstyle="round,pad=0.3", facecolor="#ffb3b3", alpha=0.9),
#                fontsize=10, ha='center')
        
#         # Region 2: a1=0, a2>0
#         ax.text(-0.7, 0.7, f'Region 2:\ny = {w_out2:.1f}×a₂ + {b_out:.1f}', 
#                bbox=dict(boxstyle="round,pad=0.3", facecolor="#b3d9d9", alpha=0.9),
#                fontsize=10, ha='center')
        
#         # Region 3: a1>0, a2>0
#         if w_out1 >= 0 and w_out2 >= 0:
#             region3_x, region3_y = 0.6, 0.6
#         elif w_out1 < 0 and w_out2 >= 0:
#             region3_x, region3_y = -0.3, 0.6
#         elif w_out1 >= 0 and w_out2 < 0:
#             region3_x, region3_y = 0.6, -0.3
#         else:
#             region3_x, region3_y = 0.3, 0.3
            
#         ax.text(region3_x, region3_y, f'Region 3:\ny = {w_out1:.1f}×a₁ + {w_out2:.1f}×a₂ + {b_out:.1f}', 
#                bbox=dict(boxstyle="round,pad=0.3", facecolor="#d9b3ff", alpha=0.9),
#                fontsize=10, ha='center')
    
#     def plot_decision_boundary(self, ax, w1, w2, b, color, label, linewidth=4, alpha=0.9):
#         """Plot decision boundary where w1*x1 + w2*x2 + b = 0"""
#         if abs(w2) > 1e-6:  # Not vertical line
#             x1_vals = np.linspace(-1, 1, 100)
#             x2_vals = -(w1 * x1_vals + b) / w2
#             # Only plot within bounds
#             mask = (x2_vals >= -1) & (x2_vals <= 1)
#             if np.any(mask):
#                 ax.plot(x1_vals[mask], x2_vals[mask], 
#                        color=color, linewidth=linewidth, label=label, alpha=alpha)
#         elif abs(w1) > 1e-6:  # Vertical line
#             x1_boundary = -b / w1
#             if -1 <= x1_boundary <= 1:
#                 ax.axvline(x1_boundary, color=color, linewidth=linewidth, 
#                           label=label, alpha=alpha)
    
#     def reset_values(self, button):
#         self.w11_slider.value = 1.0
#         self.w12_slider.value = 1.0
#         self.b1_slider.value = 0.0
#         self.w21_slider.value = -1.0
#         self.w22_slider.value = 1.0
#         self.b2_slider.value = 0.0
#         self.w_out1_slider.value = 1.0
#         self.w_out2_slider.value = -1.0
#         self.b_out_slider.value = 0.0
        
#     def display(self):
#         # Create layout
#         controls = widgets.HBox([
#             self.hidden_layer_box,
#             self.output_layer_box
#         ], layout=widgets.Layout(justify_content='space-around'))
        
#         buttons = widgets.HBox([
#             self.auto_update,
#             self.update_button,
#             self.reset_button
#         ], layout=widgets.Layout(justify_content='center'))
        
#         # Display everything
#         display(widgets.VBox([
#             widgets.HTML('<h2 style="text-align: center; color: #2c3e50; margin-bottom: 15px;">🧠 Neural Network: Hidden + Output Layer Visualization</h2>'),
#             controls,
#             widgets.HTML('<br>'),
#             buttons,
#             widgets.HTML('<hr style="margin: 15px 0;">'),
#             self.output
#         ]))
        
#         # Initial plot
#         self.update_plot()

# # Create and display the visualizer
# print("🚀 Creating 2-Layer Neural Network Visualizer...")
# visualizer = NeuralNetworkVisualizer()
# visualizer.display()

# print("""
# 🧠 KEY INSIGHT: THE OUTPUT LAYER USES THE SAME OPERATION EVERYWHERE!

# 🎯 What you're seeing:
# • LEFT PLOT: Hidden layer creates 4 distinct regions with different (a₁, a₂) values
# • RIGHT PLOT: Output layer applies SAME formula y = w₁×a₁ + w₂×a₂ + b to ALL regions
# • Result: Piecewise linear function (different slopes in each region)

# 🔍 THE CRITICAL UNDERSTANDING:
# ❌ WRONG: "Little copies of different operations in each region"
# ✅ CORRECT: "Same operation applied to different input values (a₁, a₂)"

# 📊 In each region:
# • Region 0: a₁=0, a₂=0 → y = b_out (constant)
# • Region 1: a₁>0, a₂=0 → y = w_out1×a₁ + b_out (linear in a₁)
# • Region 2: a₁=0, a₂>0 → y = w_out2×a₂ + b_out (linear in a₂)  
# • Region 3: a₁>0, a₂>0 → y = w_out1×a₁ + w_out2×a₂ + b_out (bilinear)

# 🎛️ EXPERIMENTS:
# 1. Set w_out1=1, w_out2=0 → output only depends on neuron 1
# 2. Change output weights → see how slopes change in each region
# 3. Note: boundaries stay the same (determined by hidden layer only!)

# 💡 This is how deep networks build complex functions: each layer applies simple operations to increasingly complex representations!
# """)

🚀 Creating 2-Layer Neural Network Visualizer...



🧠 KEY INSIGHT: THE OUTPUT LAYER USES THE SAME OPERATION EVERYWHERE!

🎯 What you're seeing:
• LEFT PLOT: Hidden layer creates 4 distinct regions with different (a₁, a₂) values
• RIGHT PLOT: Output layer applies SAME formula y = w₁×a₁ + w₂×a₂ + b to ALL regions
• Result: Piecewise linear function (different slopes in each region)

🔍 THE CRITICAL UNDERSTANDING:
❌ WRONG: "Little copies of different operations in each region"
✅ CORRECT: "Same operation applied to different input values (a₁, a₂)"

📊 In each region:
• Region 0: a₁=0, a₂=0 → y = b_out (constant)
• Region 1: a₁>0, a₂=0 → y = w_out1×a₁ + b_out (linear in a₁)
• Region 2: a₁=0, a₂>0 → y = w_out2×a₂ + b_out (linear in a₂)  
• Region 3: a₁>0, a₂>0 → y = w_out1×a₁ + w_out2×a₂ + b_out (bilinear)

🎛️ EXPERIMENTS:
1. Set w_out1=1, w_out2=0 → output only depends on neuron 1
2. Change output weights → see how slopes change in each region
3. Note: boundaries stay the same (determined by hidden layer only!)

💡 This is how deep networks bui

In [9]:
# # Neural Network Input Space Partitioning Visualization
# # This version works with standard Jupyter installations

# import numpy as np
# import matplotlib.pyplot as plt
# from matplotlib.colors import LinearSegmentedColormap
# import ipywidgets as widgets
# from IPython.display import display, clear_output
# import matplotlib.patches as patches

# class NeuralNetworkVisualizer:
#     def __init__(self):
#         self.output = widgets.Output()
#         self.setup_widgets()
        
#     def setup_widgets(self):
#         # Create sliders for weights and biases
#         style = {'description_width': '130px'}
#         layout = widgets.Layout(width='320px')
        
#         # Neuron 1 controls
#         self.w11_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₁₁ (Neuron 1):', style=style, layout=layout
#         )
#         self.w12_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₁₂ (Neuron 1):', style=style, layout=layout
#         )
#         self.b1_slider = widgets.FloatSlider(
#             value=0.0, min=-3.0, max=3.0, step=0.1,
#             description='b₁ (Neuron 1):', style=style, layout=layout
#         )
        
#         # Neuron 2 controls
#         self.w21_slider = widgets.FloatSlider(
#             value=-1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₂₁ (Neuron 2):', style=style, layout=layout
#         )
#         self.w22_slider = widgets.FloatSlider(
#             value=1.0, min=-3.0, max=3.0, step=0.1,
#             description='w₂₂ (Neuron 2):', style=style, layout=layout
#         )
#         self.b2_slider = widgets.FloatSlider(
#             value=0.0, min=-3.0, max=3.0, step=0.1,
#             description='b₂ (Neuron 2):', style=style, layout=layout
#         )
        
#         # Reset button
#         self.reset_button = widgets.Button(
#             description='Reset to Defaults',
#             button_style='info',
#             layout=widgets.Layout(width='200px')
#         )
        
#         # Auto-update checkbox
#         self.auto_update = widgets.Checkbox(
#             value=True,
#             description='Auto-update plot',
#             style={'description_width': 'initial'}
#         )
        
#         # Manual update button
#         self.update_button = widgets.Button(
#             description='Update Plot',
#             button_style='success',
#             layout=widgets.Layout(width='150px')
#         )
        
#         # Group controls
#         self.neuron1_box = widgets.VBox([
#             widgets.HTML('<h3 style="color: #ff6b6b; margin: 5px 0;">🔴 Neuron 1 (Red)</h3>'),
#             widgets.HTML('<div style="font-family: monospace; background: #f0f0f0; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 12px;">z₁ = w₁₁×x₁ + w₁₂×x₂ + b₁<br>a₁ = ReLU(z₁) = max(0, z₁)</div>'),
#             self.w11_slider,
#             self.w12_slider,
#             self.b1_slider
#         ])
        
#         self.neuron2_box = widgets.VBox([
#             widgets.HTML('<h3 style="color: #4ecdc4; margin: 5px 0;">🟢 Neuron 2 (Teal)</h3>'),
#             widgets.HTML('<div style="font-family: monospace; background: #f0f0f0; padding: 8px; margin: 5px 0; border-radius: 4px; font-size: 12px;">z₂ = w₂₁×x₁ + w₂₂×x₂ + b₂<br>a₂ = ReLU(z₂) = max(0, z₂)</div>'),
#             self.w21_slider,
#             self.w22_slider,
#             self.b2_slider
#         ])
        
#         # Bind events
#         for slider in [self.w11_slider, self.w12_slider, self.b1_slider,
#                       self.w21_slider, self.w22_slider, self.b2_slider]:
#             slider.observe(self.on_value_change, names='value')
        
#         self.reset_button.on_click(self.reset_values)
#         self.update_button.on_click(self.manual_update)
        
#     def relu(self, x):
#         return np.maximum(0, x)
    
#     def on_value_change(self, change):
#         if self.auto_update.value:
#             self.update_plot()
    
#     def manual_update(self, button=None):
#         self.update_plot()
    
#     def update_plot(self):
#         with self.output:
#             clear_output(wait=True)
            
#             # Get current values
#             w11, w12, b1 = self.w11_slider.value, self.w12_slider.value, self.b1_slider.value
#             w21, w22, b2 = self.w21_slider.value, self.w22_slider.value, self.b2_slider.value
            
#             # Create figure
#             fig, ax = plt.subplots(figsize=(6, 6))
            
#             # Create grid
#             x1 = np.linspace(-1, 1, 300)
#             x2 = np.linspace(-1, 1, 300)
#             X1, X2 = np.meshgrid(x1, x2)
            
#             # Calculate neuron outputs
#             z1 = w11 * X1 + w12 * X2 + b1
#             z2 = w21 * X1 + w22 * X2 + b2
            
#             a1 = self.relu(z1)
#             a2 = self.relu(z2)
            
#             # Create region map
#             region_map = np.zeros_like(X1)
            
#             # Define regions
#             mask1 = (a1 > 0) & (a2 == 0)  # Only neuron 1 active
#             mask2 = (a1 == 0) & (a2 > 0)  # Only neuron 2 active  
#             mask3 = (a1 > 0) & (a2 > 0)   # Both neurons active
#             # mask0 = (a1 == 0) & (a2 == 0) # No neurons active (default 0)
            
#             region_map[mask1] = 1
#             region_map[mask2] = 2
#             region_map[mask3] = 3
            
#             # Custom colormap
#             colors = ['#f8f9fa', '#ff9999', '#66d9d9', '#cc99ff']
#             cmap = LinearSegmentedColormap.from_list('custom', colors, N=4)
            
#             # Plot regions
#             im = ax.imshow(region_map, extent=[-1, 1, -1, 1], 
#                           cmap=cmap, alpha=0.8, origin='lower')
            
#             # Plot decision boundaries
#             self.plot_decision_boundary(ax, w11, w12, b1, 'red', 'Neuron 1 boundary')
#             self.plot_decision_boundary(ax, w21, w22, b2, 'cyan', 'Neuron 2 boundary')
            
#             # Add contour lines for activation
#             if np.max(a1) > 0.1:
#                 levels1 = np.linspace(0.2, np.max(a1), 6)
#                 contour1 = ax.contour(X1, X2, a1, levels=levels1, colors='darkred', 
#                                      alpha=0.6, linewidths=1.5)
#                 ax.clabel(contour1, inline=True, fontsize=8, fmt='%.1f')
            
#             if np.max(a2) > 0.1:
#                 levels2 = np.linspace(0.2, np.max(a2), 6)
#                 contour2 = ax.contour(X1, X2, a2, levels=levels2, colors='darkcyan', 
#                                      alpha=0.6, linewidths=1.5)
#                 ax.clabel(contour2, inline=True, fontsize=8, fmt='%.1f')
            
#             # Add gradient arrows
#             self.add_gradient_arrows(ax, w11, w12, w21, w22)
            
#             # Formatting
#             ax.set_xlim(-1, 1)
#             ax.set_ylim(-1, 1)
#             ax.set_xlabel('x₁ (Input 1)', fontsize=14)
#             ax.set_ylabel('x₂ (Input 2)', fontsize=14)
#             ax.set_title('Neural Network Input Space Partitioning', fontsize=16, pad=20)
#             ax.grid(True, alpha=0.3, linewidth=0.5)
#             ax.set_aspect('equal')
            
#             # Add current equations
#             eq1 = f'z₁ = {w11:.1f}×x₁ + {w12:.1f}×x₂ + {b1:.1f}'
#             eq2 = f'z₂ = {w21:.1f}×x₁ + {w22:.1f}×x₂ + {b2:.1f}'
            
#             ax.text(0.02, 0.98, eq1, fontsize=11, weight='bold',
#                    bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.9),
#                    transform=ax.transAxes, verticalalignment='top')
#             ax.text(0.02, 0.90, eq2, fontsize=11, weight='bold',
#                    bbox=dict(boxstyle="round,pad=0.4", facecolor="white", alpha=0.9),
#                    transform=ax.transAxes, verticalalignment='top')
            
#             # Legend
#             legend_elements = [
#                 patches.Patch(color='#ff9999', label='🔴 Neuron 1 active only'),
#                 patches.Patch(color='#66d9d9', label='🟢 Neuron 2 active only'),
#                 patches.Patch(color='#cc99ff', label='🟣 Both neurons active'),
#                 patches.Patch(color='#f8f9fa', label='⚪ No neurons active')
#             ]
#             ax.legend(handles=legend_elements, loc='upper right', 
#                      bbox_to_anchor=(1.0, 1.0), framealpha=0.95)
            
#             plt.tight_layout()
#             plt.show()
    
#     def plot_decision_boundary(self, ax, w1, w2, b, color, label):
#         """Plot decision boundary where w1*x1 + w2*x2 + b = 0"""
#         if abs(w2) > 1e-6:  # Not vertical line
#             x1_vals = np.linspace(-1, 1, 100)
#             x2_vals = -(w1 * x1_vals + b) / w2
#             # Only plot within bounds
#             mask = (x2_vals >= -1) & (x2_vals <= 1)
#             if np.any(mask):
#                 ax.plot(x1_vals[mask], x2_vals[mask], 
#                        color=color, linewidth=4, label=label, alpha=0.9)
#         elif abs(w1) > 1e-6:  # Vertical line
#             x1_boundary = -b / w1
#             if -1 <= x1_boundary <= 1:
#                 ax.axvline(x1_boundary, color=color, linewidth=4, 
#                           label=label, alpha=0.9)
    
#     def add_gradient_arrows(self, ax, w11, w12, w21, w22):
#         """Add arrows showing gradient direction"""
#         arrow_scale = 0.15
        
#         # Neuron 1 gradient arrow
#         if abs(w11) > 1e-6 or abs(w12) > 1e-6:
#             norm1 = np.sqrt(w11**2 + w12**2)
#             ax.arrow(0.1, 0.1, w11*arrow_scale/norm1, w12*arrow_scale/norm1, 
#                     head_width=0.03, head_length=0.03, 
#                     fc='red', ec='red', alpha=0.8, linewidth=2)
        
#         # Neuron 2 gradient arrow  
#         if abs(w21) > 1e-6 or abs(w22) > 1e-6:
#             norm2 = np.sqrt(w21**2 + w22**2)
#             ax.arrow(-0.1, -0.1, w21*arrow_scale/norm2, w22*arrow_scale/norm2, 
#                     head_width=0.03, head_length=0.03, 
#                     fc='cyan', ec='cyan', alpha=0.8, linewidth=2)
    
#     def reset_values(self, button):
#         self.w11_slider.value = 1.0
#         self.w12_slider.value = 1.0
#         self.b1_slider.value = 0.0
#         self.w21_slider.value = -1.0
#         self.w22_slider.value = 1.0
#         self.b2_slider.value = 0.0
        
#     def display(self):
#         # Create layout
#         controls = widgets.HBox([
#             self.neuron1_box,
#             self.neuron2_box
#         ], layout=widgets.Layout(justify_content='space-around'))
        
#         buttons = widgets.HBox([
#             self.auto_update,
#             self.update_button,
#             self.reset_button
#         ], layout=widgets.Layout(justify_content='center'))
        
#         # Display everything
#         display(widgets.VBox([
#             widgets.HTML('<h2 style="text-align: center; color: #2c3e50; margin-bottom: 15px;">🧠 Neural Network Input Space Partitioning</h2>'),
#             controls,
#             widgets.HTML('<br>'),
#             buttons,
#             widgets.HTML('<hr style="margin: 15px 0;">'),
#             self.output
#         ]))
        
#         # Initial plot
#         self.update_plot()

# # Create and display the visualizer
# print("🚀 Creating Neural Network Visualizer...")
# visualizer = NeuralNetworkVisualizer()
# visualizer.display()

# print("""
# 📚 VISUALIZATION GUIDE:

# 🎯 REGIONS:
# 🔴 RED: Only Neuron 1 active (a₁ > 0, a₂ = 0)
# 🟢 TEAL: Only Neuron 2 active (a₁ = 0, a₂ > 0)  
# 🟣 PURPLE: Both neurons active (a₁ > 0, a₂ > 0)
# ⚪ WHITE: No neurons active (a₁ = 0, a₂ = 0)

# 🔍 VISUAL ELEMENTS:
# • Thick colored lines = Decision boundaries (where z = 0)
# • Thin contour lines = Activation magnitude levels
# • Arrows = Gradient direction (perpendicular to boundary)
# • Equations = Current neuron formulas

# 🎛️ CONTROLS:
# • Auto-update: Real-time updates as you drag sliders
# • Manual update: Click button to refresh plot
# • Reset: Return to default values

# 🧪 EXPERIMENTS TO TRY:
# 1. Set both biases to 0 → see pure weight effects
# 2. Make w₁₁ = w₂₁, w₁₂ = w₂₂ → parallel boundaries  
# 3. Set w₁₂ = w₂₂ = 0 → vertical boundaries
# 4. Vary biases with fixed weights → boundary translation
# 5. Try w₁₁ = 1, w₁₂ = 0, w₂₁ = 0, w₂₂ = 1 → axis-aligned regions

# 💡 This shows how neural networks partition input space into linear regions!
# """)